In [386]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns

In [364]:
data=pd.read_csv("data.xlsx - Sheet1.csv")

In [365]:
data.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [366]:
X = data.drop('PE',axis=1).values
Y = data['PE'].values

In [367]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [368]:
x_train.shape , x_test.shape , y_train.shape , y_test.shape 

((7654, 4), (1914, 4), (7654,), (1914,))

In [369]:
sca = StandardScaler()
sca.fit(X)
x_train = sca.transform(x_train)
x_test= sca.transform(x_test)

In [370]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)





In [371]:
class ANN(nn.Module):
    def __init__(self,input_features=4,hidden1=16,hidden2=32,out_features=1):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,out_features)         
    def forward(self,x):
        x=torch.relu(self.f_connected1(x))
        x=torch.relu(self.f_connected2(x))
        x=self.out(x)
        return x
        

In [372]:
torch.manual_seed(4)
model=ANN()

In [373]:
model.parameters

<bound method Module.parameters of ANN(
  (f_connected1): Linear(in_features=4, out_features=16, bias=True)
  (f_connected2): Linear(in_features=16, out_features=32, bias=True)
  (out): Linear(in_features=32, out_features=1, bias=True)
)>

In [374]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
epochs = 200
train_loss=[]
test_loss = []
for i in range(epochs):
    i=i+1
    y_pred = model(x_train)
    loss = loss_func(y_pred,y_train)
    train_loss.append(loss)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
        output=model(x_test)
        loss = loss_func(output,y_test)
        test_loss.append(loss)